In [1]:
!pip install torch torchvision transformers diffusers bitsandbytes imageio av gradiog

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import av
import numpy as np
import torch
import gc
import imageio
from transformers import (
    AutoImageProcessor,
    AutoTokenizer,
    VisionEncoderDecoderModel,
    T5EncoderModel,
    BitsAndBytesConfig,
)
from diffusers import LattePipeline
import gradio as gr

# Set up device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Function to clear GPU cache
def flush():
    gc.collect()
    torch.cuda.empty_cache()

# Load models for video captioning
image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
caption_model = VisionEncoderDecoderModel.from_pretrained(
    "Neleac/timesformer-gpt2-video-captioning"
).to(device)

# Define the main processing function
def process_video(video_input):
    # Handle Gradio video input
    if isinstance(video_input, dict):
        video_path = video_input["name"]
    else:
        video_path = video_input  # Assume it's a filepath

    # Step 1: Generate caption from the input video
    container = av.open(video_path)

    # Extract evenly spaced frames from the video
    seg_len = container.streams.video[0].frames
    clip_len = caption_model.config.encoder.num_frames
    indices = set(
        np.linspace(0, seg_len - 1, num=clip_len, endpoint=True).astype(np.int64)
    )
    frames = []
    container.seek(0)
    for i, frame in enumerate(container.decode(video=0)):
        if i in indices:
            frames.append(frame.to_ndarray(format="rgb24"))
    container.close()

    # Generate caption
    gen_kwargs = {
        "min_length": 10,
        "max_length": 20,
        "num_beams": 5,
        "no_repeat_ngram_size": 2,
    }
    pixel_values = image_processor(frames, return_tensors="pt").pixel_values.to(device)
    tokens = caption_model.generate(pixel_values, **gen_kwargs)
    caption = tokenizer.decode(tokens[0], skip_special_tokens=True)

    # Step 2: Use the caption to generate a video
    prompt = caption
    negative_prompt = ""

    # Initialize the text encoder
    text_encoder = T5EncoderModel.from_pretrained(
        "maxin-cn/Latte-1",
        subfolder="text_encoder",
        quantization_config=BitsAndBytesConfig(
            load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16
        ),
        device_map="auto",
    )

    # Initialize the pipeline with the text encoder
    pipe = LattePipeline.from_pretrained(
        "maxin-cn/Latte-1",
        text_encoder=text_encoder,
        transformer=None,
        device_map="balanced",
    )

    # Encode the prompt
    with torch.no_grad():
        prompt_embeds, negative_prompt_embeds = pipe.encode_prompt(
            prompt, negative_prompt=negative_prompt
        )

    # Release text encoder and pipeline to free memory
    del text_encoder
    del pipe
    flush()

    # Reload the pipeline without the text encoder
    pipe = LattePipeline.from_pretrained(
        "maxin-cn/Latte-1",
        text_encoder=None,
        torch_dtype=torch.float16,
    ).to(device)

    # Generate the video
    with torch.no_grad():
        videos = pipe(
            video_length=16,
            num_inference_steps=50,
            negative_prompt=None,
            prompt_embeds=prompt_embeds,
            negative_prompt_embeds=negative_prompt_embeds,
            output_type="pt",
        ).frames.cpu()

    # Save the generated video
    output_video_path = "generated_video.mp4"
    videos = (videos.clamp(0, 1) * 255).to(dtype=torch.uint8)
    imageio.mimwrite(
        output_video_path, videos[0].permute(0, 2, 3, 1), fps=8, quality=5
    )

    # Release pipeline and flush memory
    del pipe
    flush()

    # Return both the caption and the generated video path
    return caption, output_video_path

# Set up Gradio Interface
iface = gr.Interface(
    fn=process_video,
    inputs=gr.Video(label="Upload a Video"),
    outputs=[
        gr.Textbox(label="Generated Caption"),
        gr.Video(label="Generated Video"),
    ],
    title="Video Captioning and Generation",
    description="Upload a video to generate a caption and then generate a new video based on the caption.",
)

# Launch the Gradio App
iface.launch(debug=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/4.23G [00:00<?, ?B/s]

transformer/config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of the model checkpoint were not used when initializing LatteTransformer3DModel: 
 ['caption_projection.y_embedding']


  0%|          | 0/50 [00:00<?, ?it/s]